# [Alpha Vantage](https://www.alphavantage.co/documentation/#news-sentiment)
Limitations:
* 25 calls/day (they seem to block based on fingerprint, not key, so just change runtime on Colab for each ticker)
* max 1000 results/call

In [4]:
#arguments

av_key = 'LFZXFLHI45IN7AXV'
av_ticker = 'CRYPTO:ETH' #'CRYPTO:BTC' 'CRYPTO:ETH' 'CRYPTO:SOL'
av_from = '20120101T0000' #YYYYMMDDTHHMM
av_to = '' #optional
av_limit = 1000

In [5]:
#collect

import pandas as pd
import requests
from itertools import count

av_df = pd.DataFrame()

#first call
url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={av_ticker}&apikey={av_key}&time_from={av_from}&time_to={av_to}&limit={av_limit}'
av_data = requests.get(url).json()
av_feed = [{**item, 'ticker': av_ticker.split(':')[-1]} for item in av_data['feed']] #error if rate limit reached
print('\n' + av_ticker, end=': 1 ')
av_df = pd.concat([av_df, pd.DataFrame(av_feed)], ignore_index=True)

#subsequent calls
for av_call in count(2):
    av_last = av_df['time_published'].iat[-1][:-2] #trims seconds
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={av_ticker}&apikey={av_key}&time_from={av_from}&time_to={av_last}&limit={av_limit}'
    av_data = requests.get(url).json()
    if not 'feed' in av_data:
        print('rate limit reached', end='')
        break
    print(av_call, end=' ')
    if not av_data['feed']:
        break
    av_feed = [{**item, 'ticker':av_ticker.split(':')[-1]} for item in av_data['feed'][1:]] #skips first one
    av_df = pd.concat([av_df, pd.DataFrame(av_feed)], ignore_index=True)


CRYPTO:BTC: 1 2 3 4 5 6 7 8 rate limit reached

KeyError: 'feed'

In [42]:
#show

av_df

In [40]:
#save

av_df[['ticker', 'title', 'summary']].rename(columns={'title': 'headline', 'summary': 'preview'}).to_csv(av_ticker.lower()+'.csv', index=False)

# [Finnhub](https://finnhub.io/docs/api/company-news)
Limitations:
* 60 calls/minute
* max 1 year of historical data
* US companies only (no crypto etc.)

In [1]:
#arguments

fh_key = 'cqnr07hr01qo8864qbpgcqnr07hr01qo8864qbq0'
fh_tickers = ['AAPL', 'META'] #TODO select stocks
fh_from = '2023-01-01' #YYYY-MM-DD
fh_to = '2025-01-01'

In [2]:
#collect

import finnhub
import pandas as pd
from itertools import count
from time import sleep

client = finnhub.Client(fh_key)
fh_df = pd.DataFrame()
for fh_ticker in fh_tickers:

    #first call
    fh_data = client.company_news(fh_ticker, fh_from, fh_to)
    print('\n' + fh_ticker, end=': 1 ')
    fh_df = pd.concat([fh_df, pd.DataFrame(fh_data)], ignore_index=True)

    #subsequent calls
    for fh_call in count(2):
        sleep(1)
        fh_last = pd.Timestamp(fh_df['datetime'].iat[-1] - 86400, unit='s').strftime('%Y-%m-%d') #day before
        fh_data = client.company_news(fh_ticker, fh_from, fh_last)
        print(fh_call, end=' ')
        if not fh_data:
            break
        fh_df = pd.concat([fh_df, pd.DataFrame(fh_data)], ignore_index=True)

fh_df['datetime'] = pd.to_datetime(fh_df['datetime'], unit='s')


AAPL: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 
META: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

In [43]:
#show

fh_df

,category,datetime,headline,id,image,related,source,summary,url
0,company,2024-09-10 09:15:00,Billionaires Are Buying These 2 Top Artificial...,129799736,https://g.foolcdn.com/editorial/images/789992/...,AAPL,Yahoo,These billionaire favorites can grow even high...,https://finnhub.io/api/news?id=a89e3b8068e2255...
1,company,2024-09-10 09:08:33,"EU's Vestager: Need to Own Wins, Losses on Big...",129801614,,AAPL,Finnhub,-- The European Union's competition chief Marg...,https://finnhub.io/api/news?id=377ce114d1da6f2...
2,company,2024-09-10 08:55:00,Apple Loses Court Battle Over $14 Billion Iris...,129799737,,AAPL,Yahoo,Apple and Alphabet’s Google lost attempts to o...,https://finnhub.io/api/news?id=654af8e3f4beee7...
3,company,2024-09-10 08:47:00,29.5% of Warren Buffett's $305.7 Billion Portf...,129799738,https://g.foolcdn.com/editorial/images/790050/...,AAPL,Yahoo,Many of the stocks in Berkshire Hathaway's por...,https://finnhub.io/api/news?id=a2f2121d246d1fa...
4,company,2024-09-10 08:30:43,Apple loses EU top court case over $14.4B tax ...,129799739,https://s.yimg.com/ny/api/res/1.2/59PT5I9Dq9Mw...,AAPL,Yahoo,(Bloomberg) -- Apple Inc. lost its court fight...,https://finnhub.io/api/news?id=b183ae4a09f3148...
...,...,...,...,...,...,...,...,...,...
25464,company,2023-09-17 05:23:00,"Tech Stock Report: AAPL, AMZN, NVDA, TSLA, GOO...",122642627,,META,TalkMarkets,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=8d56670800a6add...
25465,company,2023-09-16 13:30:29,What Are FAANG Stocks?,122625498,https://media.zenfs.com/en/kiplinger.com/447cd...,META,Yahoo,"The ""FAANG"" acronym is used to describe a grou...",https://finnhub.io/api/news?id=4219ff21cd61e8d...
25466,company,2023-09-16 08:30:00,Unity: Plunged More Than 30% Recently - But He...,122624784,https://static.seekingalpha.com/cdn/s3/uploads...,META,SeekingAlpha,Unity's stock fell into a bear market recently...,https://finnhub.io/api/news?id=4a2489e7694fbaa...
25467,company,2023-09-16 06:18:00,‘Is my husband the cheapest man in America?’ W...,122626071,,META,MarketWatch,Looking for stock market analysis and research...,https://finnhub.io/api/news?id=41983fc504f0efc...


In [33]:
#save

fh_df[['related', 'headline', 'summary']].rename(columns={'related': 'ticker', 'summary': 'preview'}).to_csv('stocks.csv', index=False)